In [2]:
import numpy as np
import time
from PIL import Image

from padma.gym.exemplos_base.dados import db, generate_batch
from padma.models.encoder.encoder import EncoderModel, SIZE

encoder = EncoderModel()

2019-03-23 21:04:58,432 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  /home/ivan/pybr/ajna/padma/ajna_commons/flask/error.log
Fazendo log de atividade no arquivo  /home/ivan/pybr/ajna/padma/ajna_commons/flask/access.log


Using TensorFlow backend.


In [3]:
batch_size = 10
limit = 100
TEMBBOX = {'metadata.contentType': 'image/jpeg',
          'metadata.predictions.bbox': {'$exists': True}}
dict_filtro = TEMBBOX
# dict_filtro['metadata.predictions.index'] = False

In [4]:
db.fs.files.count_documents(dict_filtro)

33879

In [69]:
batch_gen = generate_batch(filtro=dict_filtro, batch_size=batch_size, limit=limit)

In [70]:
X = np.zeros((batch_size, *SIZE, 1), dtype=np.float32)
_ids = []
y = []
s = time.time()
for batch, rows in batch_gen:
    if len(batch) == 0:
        break
    for i, (image, row) in enumerate(zip(batch, rows)):
        image_array = encoder.image_prepare(image)
        s0 = time.time()
        X[i, :, :, :] = image_array
        _ids.append(row['_id'])
    s1 = time.time()
    print('Montou X em %0.2f ' % (s1 - s0))
    indexes = encoder.model.predict(X)
    indexes = indexes.reshape(-1, 128).astype(np.float32)
    y.append(indexes)
    s2 = time.time()
    print('Fez predição em %s' % (s2 - s1))
    # print(indexes)
s3 = time.time()
elapsed = s3 - s
print('Tempo total: %s. Por imagem: %s' % (elapsed, elapsed / limit))
    

Montou X em 0.00 
Fez predição em 0.008397817611694336
Montou X em 0.00 
Fez predição em 0.0023431777954101562
Montou X em 0.00 
Fez predição em 0.0024902820587158203
Montou X em 0.00 
Fez predição em 0.0023267269134521484
Montou X em 0.00 
Fez predição em 0.0025589466094970703
Montou X em 0.00 
Fez predição em 0.0023987293243408203
Montou X em 0.00 
Fez predição em 0.0024366378784179688
Montou X em 0.00 
Fez predição em 0.002307891845703125
Montou X em 0.00 
Fez predição em 0.0025298595428466797
Montou X em 0.00 
Fez predição em 0.0024154186248779297
Tempo total: 0.5874814987182617. Por imagem: 0.005874814987182617


In [71]:
indexes.shape

(10, 128)

In [85]:
ystack = np.vstack(y).astype(np.float32)
ystack.shape

(100, 128)

In [86]:
len(_ids)

100

In [87]:
# TODO: Fazer Update no MongoDB

In [88]:
for i, _id in enumerate(_ids):
    index_row = db.fs.files.find_one({'_id': _id})
    new_array = np.array(index_row['metadata']['predictions'][0]['index'])
    dif = sum(np.abs(new_array - ystack[i, :]))
    if dif > 0.1:
        print(dif)

0.16012729704380035
0.1452294886112213
0.10036195069551468
0.2147565670311451
0.12993916124105453
0.11821924895048141
0.12117164582014084
0.1314297318458557
0.12159998714923859
0.16199632012285292
0.10688340733759105
0.24600034207105637
0.1060423031449318
0.11674889922142029
0.10178690403699875
0.10369426757097244
0.11182985245250165
0.10869336873292923
0.12589573115110397
0.1059815101325512
0.10817946027964354
0.10725441202521324
0.1110127866268158
0.12931932508945465
0.11754245311021805
0.10718080401420593
0.10892952978610992
0.11423969641327858
0.11291814222931862
0.10281937196850777
0.10500704497098923
0.11774805188179016


In [89]:
ystack[1, :] 

array([0.        , 1.3295323 , 0.        , 0.62208045, 0.23506837,
       0.        , 0.        , 0.60089624, 0.        , 0.8367413 ,
       0.        , 0.14311704, 0.08501484, 0.        , 0.        ,
       0.2033115 , 0.        , 0.8332071 , 0.        , 0.15773495,
       0.09196949, 0.        , 0.        , 0.20543823, 0.        ,
       1.3008422 , 0.08696446, 0.23300102, 0.6513472 , 0.        ,
       0.10353534, 0.18565336, 0.        , 1.5958971 , 0.        ,
       0.76247084, 0.1755022 , 0.        , 0.        , 0.5115882 ,
       0.        , 1.0940543 , 0.        , 0.23417377, 0.07671884,
       0.        , 0.        , 0.11120108, 0.        , 1.0807375 ,
       0.        , 0.23848534, 0.08987962, 0.        , 0.        ,
       0.10644014, 0.        , 1.3831367 , 0.17802384, 0.29247293,
       0.6047541 , 0.        , 0.0622528 , 0.08638763, 0.        ,
       1.9126328 , 0.        , 0.8066143 , 0.15624855, 0.        ,
       0.        , 0.5804049 , 0.        , 1.3734322 , 0.     

In [110]:
new_list = ystack[0, :].tolist()
index_row = db.fs.files.find_one({'_id': _ids[0]})
old_list = index_row['metadata']['predictions'][0]['index']
predictions = index_row['metadata']['predictions']

In [111]:
_ids[0]

ObjectId('5c5309281004b3779c37d3a2')

In [112]:
new_list

[0.0,
 1.245349407196045,
 0.0,
 0.8244215250015259,
 0.0,
 0.0,
 0.0,
 0.38313373923301697,
 0.0,
 1.0050204992294312,
 0.0,
 0.10921120643615723,
 0.07870806753635406,
 0.0,
 0.0,
 0.2451404631137848,
 0.0,
 1.0186642408370972,
 0.0,
 0.1666346788406372,
 0.14928963780403137,
 0.0,
 0.0,
 0.19059136509895325,
 0.0,
 1.4396945238113403,
 0.09840771555900574,
 0.1592206060886383,
 0.7334077954292297,
 0.0,
 0.15451441705226898,
 0.1804516613483429,
 0.0,
 1.644312858581543,
 0.0,
 0.9947700500488281,
 0.9024505019187927,
 0.0,
 0.0,
 0.30606919527053833,
 0.1576310694217682,
 1.8539807796478271,
 0.6755284070968628,
 0.7106199264526367,
 1.4652957916259766,
 0.9925693273544312,
 0.6048958897590637,
 0.6402589082717896,
 0.08659298717975616,
 1.835132122039795,
 0.6532987952232361,
 0.8272103667259216,
 0.6761195659637451,
 0.9225131273269653,
 0.43156689405441284,
 0.34402188658714294,
 0.30676060914993286,
 1.8281491994857788,
 0.7801331281661987,
 0.6974630355834961,
 0.6179544329643

In [113]:
old_list

[0.0,
 1.2464443445205688,
 0.0,
 0.8246548771858215,
 0.0,
 0.0,
 0.0,
 0.38457348942756653,
 0.0,
 1.0052496194839478,
 0.0,
 0.1068667620420456,
 0.07894809544086456,
 0.0,
 0.0,
 0.2448502480983734,
 0.0,
 1.016670823097229,
 0.0,
 0.1663249433040619,
 0.1495649218559265,
 0.0,
 0.0,
 0.18852409720420837,
 0.0,
 1.4383600950241089,
 0.09631088376045227,
 0.1597435623407364,
 0.7316836714744568,
 0.0,
 0.15341518819332123,
 0.18055927753448486,
 0.0,
 1.6475549936294556,
 0.0,
 0.9959424734115601,
 0.9046587347984314,
 0.0,
 0.0,
 0.3053806722164154,
 0.1578250378370285,
 1.8547816276550293,
 0.6739070415496826,
 0.7117723822593689,
 1.4663945436477661,
 0.9914004802703857,
 0.6043463945388794,
 0.6394813060760498,
 0.08423559367656708,
 1.837175726890564,
 0.6514485478401184,
 0.827265202999115,
 0.6778658628463745,
 0.9194700717926025,
 0.43227124214172363,
 0.34267476201057434,
 0.30516502261161804,
 1.828338861465454,
 0.7793667912483215,
 0.6979590654373169,
 0.6195490956306458

In [119]:
new_predictions = predictions
new_predictions[0]['index'] = new_list
new_predictions

[{'bbox': [20, 167, 151, 403],
  'class': 2,
  'index': [0.0,
   1.245349407196045,
   0.0,
   0.8244215250015259,
   0.0,
   0.0,
   0.0,
   0.38313373923301697,
   0.0,
   1.0050204992294312,
   0.0,
   0.10921120643615723,
   0.07870806753635406,
   0.0,
   0.0,
   0.2451404631137848,
   0.0,
   1.0186642408370972,
   0.0,
   0.1666346788406372,
   0.14928963780403137,
   0.0,
   0.0,
   0.19059136509895325,
   0.0,
   1.4396945238113403,
   0.09840771555900574,
   0.1592206060886383,
   0.7334077954292297,
   0.0,
   0.15451441705226898,
   0.1804516613483429,
   0.0,
   1.644312858581543,
   0.0,
   0.9947700500488281,
   0.9024505019187927,
   0.0,
   0.0,
   0.30606919527053833,
   0.1576310694217682,
   1.8539807796478271,
   0.6755284070968628,
   0.7106199264526367,
   1.4652957916259766,
   0.9925693273544312,
   0.6048958897590637,
   0.6402589082717896,
   0.08659298717975616,
   1.835132122039795,
   0.6532987952232361,
   0.8272103667259216,
   0.6761195659637451,
   0.9

In [121]:
import json
db.fs.files.update_one({'': _ids[0]}, {'$set': {'metadata.predicitions': json.dumps(new_predictions)}})

In [122]:
index_row = db.fs.files.find_one({'_id': _ids[0]})
saved_list = index_row['metadata']['predictions'][0]['index']
saved_list


[0.0,
 1.2464443445205688,
 0.0,
 0.8246548771858215,
 0.0,
 0.0,
 0.0,
 0.38457348942756653,
 0.0,
 1.0052496194839478,
 0.0,
 0.1068667620420456,
 0.07894809544086456,
 0.0,
 0.0,
 0.2448502480983734,
 0.0,
 1.016670823097229,
 0.0,
 0.1663249433040619,
 0.1495649218559265,
 0.0,
 0.0,
 0.18852409720420837,
 0.0,
 1.4383600950241089,
 0.09631088376045227,
 0.1597435623407364,
 0.7316836714744568,
 0.0,
 0.15341518819332123,
 0.18055927753448486,
 0.0,
 1.6475549936294556,
 0.0,
 0.9959424734115601,
 0.9046587347984314,
 0.0,
 0.0,
 0.3053806722164154,
 0.1578250378370285,
 1.8547816276550293,
 0.6739070415496826,
 0.7117723822593689,
 1.4663945436477661,
 0.9914004802703857,
 0.6043463945388794,
 0.6394813060760498,
 0.08423559367656708,
 1.837175726890564,
 0.6514485478401184,
 0.827265202999115,
 0.6778658628463745,
 0.9194700717926025,
 0.43227124214172363,
 0.34267476201057434,
 0.30516502261161804,
 1.828338861465454,
 0.7793667912483215,
 0.6979590654373169,
 0.6195490956306458